In [146]:
# import libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
%matplotlib inline

# 1. Preprocessing 

## A. The dataset

In [147]:
data = pd.read_csv('train.csv')

# check the shape of the data
print(data.shape)


(1460, 81)


In [148]:
# check the first few rows of the data  
print(data.head())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

### Sanity check

Before we go on and process this data, we need to be sure it actually makes sense. There are three "low hanging fruits" in this sense:
- Features that represent years should not go take values larger than 2018
- Areas, distances and prices should not take negative values
- Months should be between 1 and 12

In [149]:
years = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
metrics = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 
         'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

In [150]:
data[years].max()

YearBuilt       2010.0
YearRemodAdd    2010.0
GarageYrBlt     2010.0
YrSold          2010.0
dtype: float64

In [151]:
mask = (data[years] > 2018).any(axis=1) # take any index with a twisted year value
data[mask]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


In [152]:
data.loc[mask, 'GarageYrBlt'] = data[mask]['YearBuilt']

In [153]:
mask = (data[metrics] < 0).any(axis=1)
data[mask]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


### Data types

In terms of data type, there are four big groups:

1. Continuous numerical features: lengths, areas, prices
2. Discrete numerical features: numerical scores, number of bedrooms, years; they support order and arithmetic operations, so they can be treated as numerical
3. Ordinal categorical features: features with qualitative scores (such as 'Excellent' or 'Slightly Irregular'); They support ordering ('Gentle slope' < 'Severe slope') but not arithmetic operations (how much is 'Sever slope' - 'Gentle slope'?)
4. Purely categorical features: a few examples are 'MSSubClass' or 'SaleType'

After some trial and error, I decide to separate numerical (both continuous and discrete) from categorical.

The pros:
- We keep the relationship between discrete numerical features
- We end up with less features, hence less risk of overfitting

The cons: 
- We add some arbitrariness (if a house is an "8" is it really twice as better than a house that scores a 4?)

In [154]:
# check the data types of the columns
print(data.dtypes)


Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object


In [155]:
# check the summary statistics of the data
print(data.describe())


                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.066207   456.098091  ..

## B. Normality and skewness

Many regression models are more comfortable with normally distributed variables (or at least something close to it). 

We will, however, skip the discrete numerical features because:
- The results will be more readable this way
- Most of the discrete numerical features only take a few different values, so hoping for normality is a waste of time

In [156]:
# check the missing values in the data
print(data.isnull().sum())


Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64


In [157]:
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].median())

### KitchenQual

Again, very few missing values. We will substitute in this case with the 'OverallQual' value.

In [158]:
data['KitchenQual'].fillna(data['OverallQual'], inplace=True)

### Basement, garage, fireplaces and other features

We can interpret an NA in all these things as "the house does not have this feature".

In [159]:
bsmt = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
        'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'BsmtFullBath',
        'BsmtHalfBath', 
        'TotalBsmtSF']
fire = ['Fireplaces', 'FireplaceQu']
garage = ['GarageQual', 'GarageCond', 'GarageType', 'GarageFinish', 'GarageCars', 
          'GarageArea', 'GarageYrBlt']
masn = ['MasVnrType', 'MasVnrArea']
others = ['Alley', 'Fence', 'PoolQC', 'MiscFeature']

cats = data.columns[data.dtypes == 'object']
nums = list(set(data.columns) - set(cats))

# Be sure the category 'None' is also handled here
data['MasVnrType'].replace({'None': np.nan}, inplace=True)

data[cats] = data[cats].fillna('0')
data[nums] = data[nums].fillna(0)

In [163]:
data.isnull().sum().sum()

0

### Adjusting the type of variable

First, there are a few features that are not represented with the right type of variable:

- 'MSSubClass': represented as an integer, when it is just a category label (we will use 'object' for now)
- 'MoSold': represented as an integer, a month is just a category label out of 12 possibilities (we will use 'object' for now)
- 'BsmtFullBath', 'BsmtHalfBath': these two represent integers and not floats (or I at least I do not know what a third of half bathroom is)
- years: a year, in the context of this dataset, is an integer, and not a float
- 'GarageCars': represented as a float, it is an actual integer (nobody wants to have 0.5 car at home)

In [161]:
data['MSSubClass'] = data['MSSubClass'].astype('object', copy=False)
data['MoSold'] = data['MoSold'].astype('object', copy=False)
data['BsmtFullBath'] = data['BsmtFullBath'].astype('int64', copy=False)
data['BsmtHalfBath'] = data['BsmtHalfBath'].astype('int64', copy=False)
data['GarageCars'] = data['GarageCars'].astype('int64', copy=False)
data[years] = data[years].astype('int64', copy=False)

In [165]:
test=pd.get_dummies(data, drop_first=True)
X = test.drop('SalePrice', axis=1)
y = test['SalePrice']
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [166]:
#accuracy
model.score(X, y)

0.9344657880241951